In [ ]:
import time
import os
import scipy.stats as ss
from coffea import hist
from coffea.analysis_objects import JaggedCandidateArray
import coffea.processor as processor
from coffea import util
from awkward import JaggedArray
import numpy as np
import glob as glob
import itertools
import pandas as pd

In [ ]:
QCD_unweighted = util.load('TTbarResCoffea_QCD_unweighted_output.coffea')
QCD_weighted = util.load('TTbarResCoffea_QCD_weighted_output.coffea')

TTbar_unweighted = util.load('TTbarResCoffea_TTbar_unweighted_output.coffea')
TTbar_weighted = util.load('TTbarResCoffea_TTbar_weighted_output.coffea')

JetHT_unweighted = util.load('TTbarResCoffea_JetHT2016_Data_unweighted_output.coffea')
JetHT_weighted = util.load('TTbarResCoffea_JetHT2016_Data_weighted_output.coffea')

In [ ]:
QCD_nevts = QCD_unweighted['cutflow']['all events']
TTbar_nevts = TTbar_unweighted['cutflow']['all events'] 
JetHT_nevts = JetHT_unweighted['cutflow']['all events']
JetHT_nevts_wgt = JetHT_weighted['cutflow']['all events']

QCD_sumw = QCD_unweighted['cutflow']['sumw']
TTbar_sumw = TTbar_unweighted['cutflow']['sumw']

QCD_sumw2 = QCD_unweighted['cutflow']['sumw2']
TTbar_sumw2 = TTbar_unweighted['cutflow']['sumw2']

In [ ]:
# ---- MC events ---- #
print('TTbar events = ', TTbar_nevts)
print('QCD events =   ', QCD_nevts)
print()
# ---- Data events ---- #
print('Data events = ', JetHT_nevts)
print()

# ---- Sum w and w2 info ---- #
print('TTbar sumw =   ', TTbar_sumw)
print('QCD sumw =     ', QCD_sumw)
print('TTbar sumw2 =  ', TTbar_sumw2)
print('QCD sumw2 =    ', QCD_sumw2)
print()

In [ ]:
print("-------QCD Cutflow--------")
for i,j in QCD_unweighted['cutflow'].items():        
    print( '%20s : %12d' % (i,j) )

In [ ]:
print("-------QCD Mistag Weighted Cutflow--------")
for i,j in QCD_weighted['cutflow'].items():        
    print( '%20s : %12d' % (i,j) )

In [ ]:
def mkdir_p(mypath):
    '''Creates a directory. equivalent to using mkdir -p on the command line'''

    from errno import EEXIST
    from os import makedirs,path

    try:
        makedirs(mypath)
    except OSError as exc: # Python >2.5
        if exc.errno == EEXIST and path.isdir(mypath):
            pass
        else: raise

In [ ]:
def DoesDirectoryExist(mypath): #extra precaution (Probably overkill...)
    '''Checks to see if Directory exists before running mkdir_p'''
    import os.path
    from os import path
    
    if path.exists(mypath):
        pass
    else:
        mkdir_p(mypath)

In [ ]:
import matplotlib.colors as colors
import matplotlib.pyplot as plt
import warnings
import re # regular expressions
warnings.filterwarnings("ignore")

# ---- Reiterate categories ---- #
ttagcats = ["Pt", "at", "0t", "pret", "1t", "1t+2t", "2t", "0t+1t+2t"] # 'Pt' = t-tagged probe
btagcats = ["0b", "1b", "2b"]
ycats = ['cen', 'fwd']

list_of_cats = [ t+b+y for t,b,y in itertools.product( ttagcats, btagcats, ycats) ]

# ---- List the Histograms Here ---- #
list_of_hists = ('jetmass', 'ttbarmass', 'jetpt', 'jeteta', 'jetphi', 'jety', 'jetdy', 
                 'probept', 'probep')
list_of_hists_4vector = ('SDmass', 'tau32', 'jetmass', 'ttbarmass', 'jetpt', 'jeteta', 'jetphi', 'jety', 'jetdy')

In [ ]:
maindirectory = os.getcwd() 

In [ ]:
""" ---------------- Luminosity and Cross Sections ---------------- """
Lum     = 137190. # pb^-1 from https://twiki.cern.ch/twiki/bin/viewauth/CMS/PdmVAnalysisSummaryTable
Lum2016 = 35920.

tt_mc_entire_sumw = 858189552.
ttbar_BR = 0.45 #Calculated using PDG 2019
ttbar_xs = 1.0 #831.76*ttbar_BR  #pb Monte Carlo already includes xs in event weight!!
ttbar_sf = ttbar_xs*Lum2016/142155064.
print(ttbar_sf)

qcd_xs = 1370000000.0 #pb From https://cms-gen-dev.cern.ch/xsdb

In [ ]:
Nevts2016 = 625516390. # from dasgoclient
Nevts2017 = 410461585.
Nevts2018 = 676328827.
Nevts = Nevts2016 + Nevts2017 + Nevts2018
Nevts_sf = Nevts2016/JetHT_unweighted['cutflow']['all events']
print(Nevts_sf)

In [ ]:
stack_tag1_opts = {'marker': '.', 'markersize': 10., 'color': 'b', 'elinewidth': 1}
stack_tag2_opts = {'marker': 's', 'markersize': 5., 'color': 'g', 'elinewidth': 1}
legend_tag_opts = {'labels':['', '', 'DeepTag', 'DeepTag MD']}
stack_ttbar_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'red'}
stack_background_opts = {'alpha': 0.8, 'edgecolor':(0,0,0,0.3), 'color': 'yellow'}
stack_error_opts = {'label':'Stat. Unc.', 'hatch':'///', 'facecolor':'None', 'edgecolor':(0,0,0,.5), 'linewidth': 0}
data_err_opts = {'linestyle': 'none', 'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1}

In [ ]:
""" Testing Mod Mass Technique for Randomly Selecting Soft Drop Masses Given QCD Simulation Distribution """

QCD_hist = QCD_unweighted['jetmass'].integrate('anacat', '2t0bcen').integrate('dataset', 'QCD') #declare histogram
rawdata = QCD_hist.values() #extract counts directly
print(rawdata.values())
rawdata = [i for i in rawdata.values()][0] #present counts as numpy array

# ---- Normalize the raw histogram counts (Should be Optional) ---- #
QCD_hist_integral = np.sum(rawdata) #integrate the histogram
QCD_hist.scale(100./QCD_hist_integral) #normalize the histogram to percentage of events

normdata = QCD_hist.values() #extract normalized counts directly
QCD_data = [i for i in normdata.values()][0] #present normalized counts as numpy array
print('Normalized counts = \n', QCD_data)

bins = np.arange(510) #Re-make bins from the jetmass_axis starting with the appropriate range
QCD_bins = bins[::10] #Finish re-making bins by insuring exactly 50 bins like the jetmass_axis
print('Jetmass Bins = \n',QCD_bins)

hist.plot1d(QCD_hist, clear=True,
            fill_opts=stack_ttbar_opts, error_opts=stack_error_opts)
plt.ylabel("% of All Events")
plt.title('QCD MC 2t0bcen')

ModMass_hist_dist = ss.rv_histogram([QCD_data,QCD_bins])

print('\n Testing random variates function: \n')
# ---- Test the random variates function ---- #
RVS = ModMass_hist_dist.rvs(size=50)
print(RVS)
print('\n Average = ', np.sum(RVS)/50.)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(
    nrows = 1,
    ncols = 3,
    figsize=(20,5),
    sharex=False
)

title1 = 'Tagger Comparison'
title2 = r'deepTag and $p_T$'
title3 = r'deepTag MD and $p_T$'

Tag1_1D = JetHT_unweighted['deepTag_TvsQCD'].integrate('anacat').integrate('dataset', 'JetHT2016_Data').sum('jetpt')
Tag2_1D = JetHT_unweighted['deepTagMD_TvsQCD'].integrate('anacat').integrate('dataset', 'JetHT2016_Data').sum('jetpt')
Tag1_2D = JetHT_unweighted['deepTag_TvsQCD'].integrate('anacat').integrate('dataset', 'JetHT2016_Data')
Tag2_2D = JetHT_unweighted['deepTagMD_TvsQCD'].integrate('anacat').integrate('dataset', 'JetHT2016_Data')

deepTag_1D   = hist.plot1d(Tag1_1D, ax=ax1, 
                           error_opts=stack_tag1_opts, clear=False)
deepTagMD_1D = hist.plot1d(Tag2_1D, ax=ax1, 
                           error_opts=stack_tag2_opts, legend_opts=legend_tag_opts, clear=False)
ax1.set_xlim([0,1])
ax1.set_ylim(10**3, 10**6)
ax1.set_yscale('log')
ax1.set_title(title1)

deepTag_2D   = hist.plot2d(Tag1_2D, xaxis='tagger', ax=ax2, 
                           patch_opts={'cmap': 'Blues', 'norm':colors.LogNorm()}, clear=True) 
ax2.set_ylim([400,3000])
ax2.set_xlim(0,1)
ax2.set_ylabel(r'Jet $p_T$ [GeV]')
ax2.set_xlabel('deepTag')
ax2.set_title(title2)

deepTagMD_2D = hist.plot2d(Tag2_2D, xaxis='tagger', ax=ax3, 
                           patch_opts={'cmap': 'Greens', 'norm':colors.LogNorm()}, clear=True)
ax3.set_ylim([400,3000])
ax3.set_xlim(0,1)
ax3.set_xlabel('deepTag MD')
ax3.set_title(title3)

In [ ]:
for icat in list_of_cats:
    fig, (ax) = plt.subplots(
    nrows = 1,
    ncols = 1,
    figsize=(6,6),
    sharex=False
    )
    
    Tau32_1D = JetHT_unweighted['tau32'].integrate('anacat',icat).integrate('dataset', 'JetHT2016_Data')
    Tau32_1D_Plot = hist.plot1d(Tau32_1D, ax=ax, error_opts=stack_tag1_opts, clear=True)
    
    title = icat
    ax.set_ylabel('Events')
    ax.set_xlabel(None)
    ax.set_title(title)

In [ ]:
fig, (ax) = plt.subplots(
nrows = 1,
ncols = 1,
figsize=(6,6),
sharex=False
)

SoftDrop_1D = JetHT_unweighted['SDmass'].integrate('anacat').integrate('dataset', 'JetHT2016_Data')
SoftDrop_Plot = hist.plot1d(SoftDrop_1D, ax=ax, error_opts=stack_tag1_opts, clear=True)

title = r'$M_{SD}$'
ax.set_ylabel('Events')
#ax.set_xlabel(None)
ax.set_title(title)

In [ ]:
""" ---------------- Unweighted Data and Unweighted MC Plots (No Mistag Rates Included) ---------------- """
"""NOTE that SDmass uses the axes called jetmass, so code will get confused unless exception is made for SDmass"""
for ihist in list_of_hists_4vector:
    SaveDirectory = maindirectory + '/' + ihist + '/' # split histograms into subdirectories
    DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
    for icat in list_of_cats: 
        plt.rcParams.update({
        'font.size': 14,
        'axes.titlesize': 18,
        'axes.labelsize': 18,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12
        })
        fig, (ax, rax) = plt.subplots(
            nrows=2,
            ncols=1,
            figsize=(7,7),
            gridspec_kw={"height_ratios": (3, 1)},
            sharex=True
        )
        fig.subplots_adjust(hspace=.07)
        title = ihist + '  ' + icat
        #filename = ihist + '_' + icat + '_LinearScale.' + 'png'
        filename = ihist + '_' + icat + '.' + 'png'
        legend_labels = {'labels':['', '', '', 'QCD Bkg (Sim.)', r'$t\bar{t}$ Sim.', 'Data'],
                         'loc': 'upper right',
                         'fontsize': 'xx-small'}
        
        #---- Define Scaled Histograms by category and scale the Total----# 
        
        # ---- Start with TTbar MC ---- #
        TTbar_hist = TTbar_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'TTbar')
        MC_hist = TTbar_hist.copy() # start with TTbar MC histograms
        if ihist == 'SDmass':
            Ntt = np.sum(MC_hist.integrate('jetmass').values()) # extract number of events from histogram directly
        else:
            Ntt = np.sum(MC_hist.integrate(ihist).values())
        Ntt = [i for i in Ntt.values()] # the zeroth column entry is the number of events defined as a float
        if Ntt[0] > 0.:
            MC_hist.scale(ttbar_xs/Ntt[0]) # Normalized to ttbar xs
        else:
            MC_hist.scale(0.)
        
        # ---- Repeat above procedure for QCD ---- #
        QCD_hist = QCD_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'QCD')
        if ihist == 'SDmass':
            Nqcd = np.sum(QCD_hist.integrate('jetmass').values())
        else:
            Nqcd = np.sum(QCD_hist.integrate(ihist).values())
        Nqcd = [j for j in Nqcd.values()]
        if Nqcd[0] > 0.:
            QCD_hist.scale(qcd_xs/Nqcd[0]) # Normalized to qcd xs
        else:
            QCD_hist.scale(0.)
        
        # ---- Add QCD cross section histogram to ttbar xs histogram ---- #
        MC_hist.add(QCD_hist) # Adds Normalized MC Histograms to get Total MC
        
        # ---- Define the data histogram ---- #
        Data_hist = JetHT_unweighted[ihist].integrate('anacat', icat).integrate('dataset', 'JetHT2016_Data')
        Data_hist.scale(Nevts_sf) #scale according to number of events in dataset
        
        # ---- Extract data events and total cross section from histograms ---- #
        if ihist == 'SDmass':
            NtotalMC = np.sum(MC_hist.integrate('jetmass').values())
            NtotalData = np.sum(Data_hist.integrate('jetmass').values())
        else:
            NtotalMC = np.sum(MC_hist.integrate(ihist).values())
            NtotalData = np.sum(Data_hist.integrate(ihist).values())
        NtotalMC = [k for k in NtotalMC.values()]
        NtotalData = [l for l in NtotalData.values()]
        
        # ---- Normalize the total cross section histogram directly to the data ---- #
        if NtotalMC[0] > 0.:
            MC_hist.scale(NtotalData[0]/NtotalMC[0])
        else:
            MC_hist.scale(0.)
        
        #---- Plot Total MC (simulated QCD background)----#
        #-----------------------------------------------------------------#
        hist.plot1d(MC_hist, ax=ax, clear=False,
                    fill_opts=stack_background_opts, error_opts=stack_error_opts)
        
        #---- Plot TTbar MC for comparison ---- #
        #-----------------------------------------------------------------#
        if Ntt[0] > 0:
            TTbar_hist.scale(ttbar_sf) # Scale to Luminosity and adjust for MC generator weight
            #TTbar_hist.scale(NtotalData[0]/Ntt[0]) # Scale up to data
            #TTbar_hist.scale(35922.*0.82*831.76/(77229341 + 78006311)) #From original analysis
        else:
            TTbar_hist.scale(0.)
            
        hist.plot1d(TTbar_hist, ax=ax, clear=False,
                    fill_opts=stack_ttbar_opts, error_opts=stack_error_opts)
        
        #---- Plot Data ----#
        #-----------------------------------------------------------------#
        hist.plot1d(Data_hist, ax=ax, clear=False, 
                    error_opts=data_err_opts,
                    legend_opts=legend_labels)
        
        ax.set_ylim(bottom=0.1)
        ax.set_yscale('log')
        ax.set_ylabel('Events')
        ax.set_xlabel(None)
        ax.set_title(title)
        #leg = ax.legend(labels=["QCD Sim. Bkg", r'$t\bar{t}$ Sim.', 'Data'])
        
        
        #---- Plot Ratio ----#
        hist.plotratio(num = Data_hist, denom = MC_hist, ax = rax,
                       error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
                       unc = 'num')
        rax.set_ylabel('Data/MC')
        rax.axhline(y=1, color='k', linestyle=':')
        rax.set_ylim(0,2)
        
        if ihist == 'ttbarmass':
            rax.set_xlim(700,5000)
        if ihist == 'jetpt':
            rax.set_xlim(0,3000)
        if ihist == 'jeteta':
            rax.set_xlim(-3,3)
        if ihist == 'tau32':
            rax.set_xlim(0,1.2)
       
        #---- Labeling ----#
        Lint = str(Lum2016*.001) # Integrated Luminosity
        lumi = plt.text(1.15, 1.07, Lint[:6] + " fb$^{-1}$",
                fontsize=16,
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes
               )
        plt.savefig(SaveDirectory+filename, bbox_inches="tight")
        #print(filename + ' saved')

In [ ]:
""" ----------------------- TEMPORARY: Add up the top tag regions ----------------------- """
for ihist in list_of_hists_4vector:
    SaveDirectory = maindirectory + '/' + ihist + '/' # split histograms into subdirectories
    DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
    for icat in ('0bcen', '0bfwd', '1bcen', '1bfwd', '2bcen', '2bfwd'):
        plt.rcParams.update({
        'font.size': 14,
        'axes.titlesize': 18,
        'axes.labelsize': 18,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12
        })
        fig, ([ax_inc, ax_all], [rax_inc, rax_all]) = plt.subplots(
            nrows=2,
            ncols=2,
            figsize=(8,8),
            gridspec_kw={"height_ratios": (3, 1)},
            sharex=True #'row'
        )
        fig.subplots_adjust(hspace=.07)
        title = ihist + '  ' + icat
        #filename = ihist + '_' + icat + '_CombinedTags_LinearScale.' + 'png'
        filename = ihist + '_' + icat + '_CombinedTags.' + 'png'
        legend_labels = {'labels':['', '', '', 'QCD Bkg (Sim.)', r'$t\bar{t}$ Sim.', 'Data'],
                         'loc': 'upper right',
                         'fontsize': 'xx-small',
                         'bbox_to_anchor': (1.5, 1.0)}
        #------------------------------------------------------------------------------------------------#
        TTbar_hist0 = TTbar_unweighted[ihist].integrate('anacat', '0t'+icat).integrate('dataset', 'TTbar')
        TTbar_hist1 = TTbar_unweighted[ihist].integrate('anacat', '1t'+icat).integrate('dataset', 'TTbar')
        TTbar_hist2 = TTbar_unweighted[ihist].integrate('anacat', '2t'+icat).integrate('dataset', 'TTbar')
        TTbar_hist0.scale(ttbar_sf) # Scale to Luminosity and adjust for MC generator weight
        TTbar_hist1.scale(ttbar_sf) # Scale to Luminosity and adjust for MC generator weight
        TTbar_hist2.scale(ttbar_sf) # Scale to Luminosity and adjust for MC generator weight
        
        TTbar_hist_inc = TTbar_hist2.add(TTbar_hist1)
        TTbar_hist_all = TTbar_hist_inc.add(TTbar_hist0)
        
        MC_hist_inc = TTbar_hist_inc.copy() # start with TTbar MC histograms
        MC_hist_all = TTbar_hist_all.copy()
        
        if ihist == 'SDmass':
            Ntt_inc = np.sum(MC_hist_inc.integrate('jetmass').values()) # extract number of events from histogram directly
            Ntt_all = np.sum(MC_hist_all.integrate('jetmass').values())
        else:
            Ntt_inc = np.sum(MC_hist_inc.integrate(ihist).values())
            Ntt_all = np.sum(MC_hist_all.integrate(ihist).values())
            
        Ntt_inc = [i for i in Ntt_inc.values()] 
        Ntt_all = [i for i in Ntt_all.values()]
        
        if Ntt_inc[0] > 0.:
            MC_hist_inc.scale(ttbar_xs/Ntt_inc[0]) # Normalized to ttbar xs
        else:
            MC_hist_inc.scale(0.)
        if Ntt_all[0] > 0.:
            MC_hist_all.scale(ttbar_xs/Ntt_all[0]) # Normalized to ttbar xs
        else:
            MC_hist_all.scale(0.)
        #------------------------------------------------------------------------------------------------#
        QCD_hist0 = QCD_unweighted[ihist].integrate('anacat', '0t'+icat).integrate('dataset', 'QCD')
        QCD_hist1 = QCD_unweighted[ihist].integrate('anacat', '1t'+icat).integrate('dataset', 'QCD')
        QCD_hist2 = QCD_unweighted[ihist].integrate('anacat', '2t'+icat).integrate('dataset', 'QCD')
        
        QCD_hist_inc = QCD_hist2.add(QCD_hist1)
        QCD_hist_all = QCD_hist_inc.add(QCD_hist0)
        
        if ihist == 'SDmass':
            Nqcd_inc = np.sum(QCD_hist_inc.integrate('jetmass').values())
            Nqcd_all = np.sum(QCD_hist_all.integrate('jetmass').values())
        else:
            Nqcd_inc = np.sum(QCD_hist_inc.integrate(ihist).values())
            Nqcd_all = np.sum(QCD_hist_all.integrate(ihist).values())
            
        Nqcd_inc = [j for j in Nqcd_inc.values()]
        Nqcd_all = [j for j in Nqcd_all.values()]
        
        if Nqcd_inc[0] > 0.:
            QCD_hist_inc.scale(ttbar_xs/Nqcd_inc[0]) # Normalized to ttbar xs
        else:
            QCD_hist_inc.scale(0.)
        if Nqcd_all[0] > 0.:
            QCD_hist_all.scale(ttbar_xs/Nqcd_all[0]) # Normalized to ttbar xs
        else:
            QCD_hist_all.scale(0.)
            
        MC_hist_inc.add(QCD_hist_inc) 
        MC_hist_all.add(QCD_hist_all)
        #------------------------------------------------------------------------------------------------#
        Data_hist0 = JetHT_unweighted[ihist].integrate('anacat', '0t'+icat).integrate('dataset', 'JetHT2016_Data')
        Data_hist1 = JetHT_unweighted[ihist].integrate('anacat', '1t'+icat).integrate('dataset', 'JetHT2016_Data')
        Data_hist2 = JetHT_unweighted[ihist].integrate('anacat', '2t'+icat).integrate('dataset', 'JetHT2016_Data')
        Data_hist0.scale(Nevts_sf)
        Data_hist1.scale(Nevts_sf)
        Data_hist2.scale(Nevts_sf)
        
        Data_hist_inc = Data_hist2.add(Data_hist1)
        Data_hist_all = Data_hist_inc.add(Data_hist0)
        
        if ihist == 'SDmass':
            NtotalMC_inc = np.sum(MC_hist_inc.integrate('jetmass').values())
            NtotalMC_all = np.sum(MC_hist_all.integrate('jetmass').values())
            NtotalData_inc = np.sum(Data_hist_inc.integrate('jetmass').values())
            NtotalData_all = np.sum(Data_hist_all.integrate('jetmass').values())
        else:
            NtotalMC_inc = np.sum(MC_hist_inc.integrate(ihist).values())
            NtotalMC_all = np.sum(MC_hist_all.integrate(ihist).values())
            NtotalData_inc = np.sum(Data_hist_inc.integrate(ihist).values())
            NtotalData_all = np.sum(Data_hist_all.integrate(ihist).values())
        
        NtotalMC_inc = [k for k in NtotalMC_inc.values()]
        NtotalMC_all = [k for k in NtotalMC_all.values()]
        NtotalData_inc = [l for l in NtotalData_inc.values()]
        NtotalData_all = [l for l in NtotalData_all.values()]
        
        if NtotalMC_inc[0] > 0.:
            MC_hist_inc.scale(NtotalData_inc[0]/NtotalMC_inc[0])
        else:
            MC_hist_inc.scale(0.)
        if NtotalMC_all[0] > 0.:
            MC_hist_all.scale(NtotalData_all[0]/NtotalMC_all[0])
        else:
            MC_hist_all.scale(0.)
        #---- Plot Total MC (simulated QCD background)----#
        #-----------------------------------------------------------------#
        hist.plot1d(MC_hist_inc, ax=ax_inc, clear=False,
                    fill_opts=stack_background_opts, error_opts=stack_error_opts)
        hist.plot1d(MC_hist_all, ax=ax_all, clear=False,
                    fill_opts=stack_background_opts, error_opts=stack_error_opts)
        #---- Plot TTbar MC for comparison ---- #
        #-----------------------------------------------------------------#
        hist.plot1d(TTbar_hist_inc, ax=ax_inc, clear=False,
                    fill_opts=stack_ttbar_opts, error_opts=stack_error_opts)
        hist.plot1d(TTbar_hist_all, ax=ax_all, clear=False,
                    fill_opts=stack_ttbar_opts, error_opts=stack_error_opts)
        #---- Plot Data ----#
        #-----------------------------------------------------------------#
        hist.plot1d(Data_hist_inc, ax=ax_inc, clear=False, 
                    error_opts=data_err_opts)#,
                    #legend_opts=legend_labels)
        hist.plot1d(Data_hist_all, ax=ax_all, clear=False, 
                    error_opts=data_err_opts,
                    legend_opts=legend_labels)
        ax_inc.set_ylim(bottom=0.1)
        ax_inc.set_yscale('log')
        ax_inc.set_ylabel('Events')
        ax_inc.set_xlabel(None)
        ax_inc.set_title(title + ' 1t + 2t')
        ax_inc.legend_ = None
        ax_all.set_ylim(bottom=0.1)
        ax_all.set_yscale('log')
        ax_all.set_ylabel(None)
        ax_all.set_xlabel(None)
        ax_all.set_title(title + ' 0t + 1t + 2t')
        #---- Plot Ratio ----#
        #-----------------------------------------------------------------#
        hist.plotratio(num = Data_hist_inc, denom = MC_hist_inc, ax = rax_inc,
                       error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
                       unc = 'num')
        hist.plotratio(num = Data_hist_all, denom = MC_hist_all, ax = rax_all,
                       error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
                       unc = 'num')
        rax_inc.set_ylabel('Data/MC')
        rax_inc.set_ylim(0,2)
        rax_all.set_ylabel(None)
        rax_all.set_ylim(0,2)
        if ihist == 'ttbarmass':
            rax_inc.set_xlim(700,5000)
            rax_all.set_xlim(700,5000)
        if ihist == 'jetpt':
            rax_inc.set_xlim(0,3000)
            rax_all.set_xlim(0,3000)
        if ihist == 'jeteta':
            rax_inc.set_xlim(-3,3)
            rax_all.set_xlim(-3,3)
        if ihist == 'tau32':
            rax_inc.set_xlim(0,1.2)
            rax_all.set_xlim(0,1.2)
        #---- Labeling ----#
        #-----------------------------------------------------------------#
        Lint = str(Lum2016*.001) # Integrated Luminosity
        lumi = plt.text(1.385, 1.1, Lint[:6] + " fb$^{-1}$",
                fontsize=16,
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes
               )
        #plt.savefig(SaveDirectory+filename, bbox_inches="tight")
        #print(SaveDirectory+filename + ' saved')

In [ ]:
btag = '0b'

Num_cen = JetHT_unweighted['numerator'].integrate('anacat', 'at'+btag+'cen').integrate('dataset', 'JetHT')
Denom_cen = JetHT_unweighted['denominator'].integrate('anacat', 'at'+btag+'cen').integrate('dataset', 'JetHT')
#mistag_cen = hist.plotratio(num = Num_cen, denom = Denom_cen,
#                                error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
#                                unc = 'num')

Num_fwd = JetHT_unweighted['numerator'].integrate('anacat', 'at'+btag+'fwd').integrate('dataset', 'JetHT')
Denom_fwd = JetHT_unweighted['denominator'].integrate('anacat', 'at'+btag+'fwd').integrate('dataset', 'JetHT')
#mistag_fwd = hist.plotratio(num = Num_fwd, denom = Denom_fwd,
#                                error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
#                                unc = 'num')

Num_inc = Num_cen.add(Num_fwd)
Denom_inc = Denom_cen.add(Denom_fwd)
mistag_inclusive = hist.plotratio(num = Num_inc, denom = Denom_inc,
                                error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
                                unc = 'num')
plt.title(btag+' y inclusive')
plt.ylim(bottom = 0, top = 0.12)
plt.xlim(left = 100, right = 10000)
#mistag_inclusive.set_xscale('log')

In [ ]:
""" ---------- Comparing Unweighted to Weighted MC Probe Jets (TEST WITHOUT DATA/ BIAS TEST) ---------- """
 
plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(7,7),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True
)
fig.subplots_adjust(hspace=.07)

#---- Define Histograms ----# 
MC_hist_wgt = QCD_weighted['ttbarmass'].integrate('anacat', 'at0bcen').integrate('dataset', 'QCD')
MC_hist_unwgt = QCD_unweighted['ttbarmass'].integrate('anacat', 'at0bcen').integrate('dataset', 'QCD')

hist.plot1d(MC_hist_wgt, ax=ax, clear=True,
                    fill_opts=stack_background_opts, error_opts=stack_error_opts)
hist.plot1d(MC_hist_unwgt, ax=ax, clear=False,
                    error_opts=data_err_opts)

ax.set_yscale('log')
ax.autoscale('y')
ax.autoscale('x')#, tight=True) # doesn't look like its working...
ax.set_ylabel('Events')
ax.set_xlabel(None)
ax.set_title('Bias Test ttbarmass 0bcen')
leg = ax.legend(labels=["Bkg. Est. (QCD Sample)", "Data (QCD Sample)"])

#---- Plot Ratio ----#
hist.plotratio(num = MC_hist_unwgt, denom = MC_hist_wgt, ax = rax,
               error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
               unc = 'num')
rax.set_ylabel('Data/Bkg')
rax.set_ylim(0,2)

#---- Labeling ----#
Lint = str(Lum*.001) # Integrated Luminosity
lumi = plt.text(1.15, 1.07, "?? fb$^{-1}$",
        fontsize=16,
        horizontalalignment='right',
        verticalalignment='top',
        transform=ax.transAxes
       )

In [ ]:
lut_file = maindirectory + '/LookupTables/mistag_QCD_at0bcen.csv'
df = pd.read_csv(lut_file)
mtr = df['M(p)'].values
MMR = mtr.sum()/mtr.size
print('Mean Mistag Rate (MMR) = ', MMR)
print()

nevts = QCD_unweighted['cutflow']['at0bcen']
bkg_nevts = QCD_weighted['cutflow']['at0bcen']

print('Number of Data =       ', nevts)
print('Number of Background = ', bkg_nevts)

In [ ]:
""" ---------- Comparing Background Estimate to Unweighted MC (CLOSURE TEST with QCD MC) ---------- """
SaveDirectory = maindirectory + '/' + 'ClosureTests' + '/' # split histograms into subdirectories
DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already

#---- Histogram Window Config. ----# 
plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(7,7),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True
)
fig.subplots_adjust(hspace=.07)

#---- Useful Information ----# 
b_y = '0bcen' # b-tag category and rapidity window
lut_file = maindirectory + '/LookupTables/mistag_JetHT_at' + b_y + '.csv'
df = pd.read_csv(lut_file)
p_vals = df['p']
mtr = df['M(p)'].values
MMR = mtr.sum()/mtr.size
print('Mean Mistag Rate (MMR) = ', MMR)
#print(p_vals)
#print(mtr)

nevts = JetHT_unweighted['cutflow']['at0bcen']
bkg_nevts = JetHT_weighted['cutflow']['at0bcen']

print('Number of Data =       ', nevts)
print('Number of Background = ', bkg_nevts)
print()

#---- Define Histograms ----# 
hist_name = 'probep'
hist_wgt_anacat = 'at' + b_y
hist_unwgt_anacat = 'At' + b_y
hist_unwgt_pretag_anacat = 'at' + b_y

MC_hist_wgt = JetHT_weighted[hist_name].integrate('anacat', hist_wgt_anacat).integrate('dataset', 'JetHT')
MC_hist_unwgt = JetHT_unweighted[hist_name].integrate('anacat', hist_unwgt_anacat).integrate('dataset', 'JetHT')
MC_hist_unwgt_pretag = JetHT_unweighted[hist_name].integrate('anacat', hist_unwgt_pretag_anacat).integrate('dataset', 'JetHT')

hist.plot1d(MC_hist_wgt, ax=ax, clear=True,
                    fill_opts=stack_background_opts, error_opts=stack_error_opts)
hist.plot1d(MC_hist_unwgt, ax=ax, clear=False,
                    error_opts=data_err_opts)

d = {'p': p_vals, 'M(p)': mtr, 
     'unwgt at0bcen MC': MC_hist_unwgt_pretag.values()[()], 
     'wgt at0bcen MC': MC_hist_wgt.values()[()], 
     'At0bcen MC': MC_hist_unwgt.values()[()]}
#df1 = pd.DataFrame(data=d)
#with pd.option_context('display.max_rows', None, 'display.max_columns', None): 
#    print(df1)

ax.set_yscale('log')
ax.autoscale('y')
ax.autoscale('x')#, tight=True) # doesn't look like its working...
ax.set_ylabel('Events')
ax.set_xlabel(None)
ax.set_title('Probe Momentum ' + b_y)
leg = ax.legend(labels=["All Probe Jets (weighted)", "T-Tagged Probe Jets"])

#---- Plot Ratio ----#
hist.plotratio(num = MC_hist_unwgt, denom = MC_hist_wgt, ax = rax,
               error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
               unc = 'num')
rax.set_ylabel('Data/Bkg')
rax.set_ylim(0,2)

#---- Labeling ----#
Lint = str(Lum*.001) # Integrated Luminosity
lumi = plt.text(1.15, 1.07, "?? fb$^{-1}$",
        fontsize=16,
        horizontalalignment='right',
        verticalalignment='top',
        transform=ax.transAxes
       )
#filename = 'ClosureTest_' + b_y + '.png'
#plt.savefig(SaveDirectory+filename, bbox_inches="tight")
#print(filename + ' saved')

In [ ]:
nevts = QCD_unweighted['cutflow']['At0bcen']
bkg_nevts = QCD_weighted['cutflow']['at0bcen']

print('Number of Data =       ', nevts)
print('Number of Background = ', bkg_nevts)

In [ ]:
""" ---------- Comparing Background Estimate to Unweighted Data (Checks One Histogram) ---------- """
""" --------------- Background: t tagged probe jet from data weighted by mistag --------------- """
""" --------------- Data: Unweighted Data from the Signal Region (2t tag region) -------------- """
 
plt.rcParams.update({
'font.size': 14,
'axes.titlesize': 18,
'axes.labelsize': 18,
'xtick.labelsize': 12,
'ytick.labelsize': 12
})
fig, (ax, rax) = plt.subplots(
    nrows=2,
    ncols=1,
    figsize=(7,7),
    gridspec_kw={"height_ratios": (3, 1)},
    sharex=True
)
fig.subplots_adjust(hspace=.07)

#---- Name of Histogram to plot ----#
name = 'probep'

#---- Define Histograms ----# 
Data_hist_wgt_1 = JetHT_weighted[name].integrate('anacat', '1t0bcen').integrate('dataset', 'JetHT')
Data_hist_wgt_2 = JetHT_weighted[name].integrate('anacat', '2t0bcen').integrate('dataset', 'JetHT')
Data_hist_unwgt = JetHT_unweighted[name].integrate('anacat', '2t0bcen').integrate('dataset', 'JetHT')
Data_hist_preunwgt = JetHT_unweighted[name].integrate('anacat', '1t0bcen').integrate('dataset', 'JetHT')

#print("Only 1t weighted region = \n", Data_hist_wgt_1.values())
#print("Only 1t pre-weighted region = \n", Data_hist_preunwgt.values())
#print("2t weighted region = \n", Data_hist_wgt_2.values())
#print("2t unweighted region (Signal) = \n", Data_hist_unwgt.values())

Background = Data_hist_wgt_1.add(Data_hist_wgt_2)
Signal = Data_hist_unwgt

hist.plot1d(Background, ax=ax, clear=True,
                    fill_opts=stack_background_opts, error_opts=stack_error_opts)
hist.plot1d(Signal, ax=ax, clear=False,
                    error_opts=data_err_opts)

ax.set_yscale('log')
ax.autoscale('y')
ax.autoscale('x')#, tight=True) # doesn't look like its working...
ax.set_ylabel('Events')
ax.set_xlabel(None)
ax.set_title(name + ' 0bcen')
leg = ax.legend(labels=["Bkg Estimate", "Data"])

#---- Plot Ratio ----#
hist.plotratio(num = Signal, denom = Background, ax = rax,
               error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
               unc = 'num')
rax.set_ylabel('Data/Bkg')
rax.set_ylim(0,2)

#---- Labeling ----#
Lint = str(Lum*.001) # Integrated Luminosity
lumi = plt.text(1.15, 1.07, Lint[:6] + " fb$^{-1}$",
        fontsize=16,
        horizontalalignment='right',
        verticalalignment='top',
        transform=ax.transAxes
       )

In [ ]:
""" --------------- Background Estimates ---------- """
""" --------------- Background: t tagged probe jet from data weighted by mistag --------------- """
""" --------------- Data: Unweighted Data from the Signal Region (2t tag region) -------------- """
bcats = [
    '0bcen', '0bfwd',
    '1bcen', '1bfwd',
    '2bcen', '2bfwd'
] # Defined only for making all background estimate plots.

for ihist in list_of_hists:
    SaveDirectory = maindirectory + '/' + ihist + '/' # split histograms into subdirectories
    DoesDirectoryExist(SaveDirectory) # no need to create the directory several times if it exists already
    for icat in bcats: 
        plt.rcParams.update({
        'font.size': 14,
        'axes.titlesize': 18,
        'axes.labelsize': 18,
        'xtick.labelsize': 12,
        'ytick.labelsize': 12
        })
        fig, (ax, rax) = plt.subplots(
            nrows=2,
            ncols=1,
            figsize=(7,7),
            gridspec_kw={"height_ratios": (3, 1)},
            sharex=True
        )
        fig.subplots_adjust(hspace=.07)
        
        # ---- Needed Strings ---- #
        title = ihist + '  ' + icat
        filename = ihist + '_' + icat + '_BkgEst.' + 'png'
        bkg_cat_1 = '1t' + icat
        signal_cat = '2t' + icat
        bkg_cat_2 = signal_cat
        
        #---- Define Histograms ----# 
        Data_hist_wgt_1 = JetHT_weighted[ihist].integrate('anacat', bkg_cat_1).integrate('dataset', 'JetHT')
        Data_hist_wgt_2 = JetHT_weighted[ihist].integrate('anacat', bkg_cat_2).integrate('dataset', 'JetHT')
        Data_hist_unwgt = JetHT_unweighted[ihist].integrate('anacat', signal_cat).integrate('dataset', 'JetHT')
        
        #TTbar_hist_wgt_1 = TTbar_weighted[ihist].integrate('anacat', bkg_cat_1).integrate('dataset', 'TTbar')
        #TTbar_hist_wgt_2 = TTbar_weighted[ihist].integrate('anacat', bkg_cat_2).integrate('dataset', 'TTbar')
        #TTbar_hist_unwgt = TTbar_unweighted[ihist].integrate('anacat', signal_cat).integrate('dataset', 'TTbar')
        
        Background = Data_hist_wgt_1.add(Data_hist_wgt_2)
        Signal = Data_hist_unwgt
        #TTbar_MC = TTbar_hist_unwgt #TTbar_hist_wgt_1.add(TTbar_hist_wgt_2)
        
        # ---- Plot Histograms ---- #
        Background.scale(Nevts_sf)
        Signal.scale(Nevts_sf)
        #TTbar_MC.scale(ttbar_sf)
        
        hist.plot1d(Background, ax=ax, clear=True,
                    fill_opts=stack_background_opts, error_opts=stack_error_opts)
        hist.plot1d(Signal, ax=ax, clear=False,
                    error_opts=data_err_opts)
        #hist.plot1d(TTbar_MC, ax=ax, clear=False,
        #            fill_opts=stack_ttbar_opts)

        ax.set_yscale('log')
        ax.autoscale('y')
        ax.autoscale('x')#, tight=True) # doesn't look like its working...
        ax.set_ylabel('Events')
        ax.set_xlabel(None)
        ax.set_title(title)
        leg = ax.legend(labels=["Bkg Estimate", "Data"])

        #---- Plot Ratio ----#
        hist.plotratio(num = Signal, denom = Background, ax = rax,
                       error_opts={'marker': '.', 'markersize': 10., 'color': 'k', 'elinewidth': 1},
                       unc = 'num')
        rax.set_ylabel('Data/Bkg')
        rax.set_ylim(0,2)

        #---- Labeling ----#
        Lint = str(Lum*.001) # Integrated Luminosity
        lumi = plt.text(1.15, 1.07, Lint[:6] + " fb$^{-1}$",
                fontsize=16,
                horizontalalignment='right',
                verticalalignment='top',
                transform=ax.transAxes
               )
        #plt.savefig(SaveDirectory+filename, bbox_inches="tight")
        #print(filename + ' saved')